In [1]:
! pip install mysqlclient

You should consider upgrading via the 'c:\users\administrator\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import MySQLdb
db = MySQLdb.connect('localhost','root','1234','kdt')
cur = db.cursor()
sql = 'create table if not exists  student(num varchar(20) primary key, name varchar(20) not null, hp varchar(20) not null, email varchar(20) not null, address varchar(20) not null ,enterdate date default(current_date))'
cur.execute(sql)
db.commit()
cur.close()
# 학생 정보테이블 생성 

cur = db.cursor()
sql = 'create table if not exists  grade(num varchar(20), java int(10) default 0, python int(10) default 0, c int(10) default 0, total int(20) default 0, avg int(20) default 0, enterdate date default (current_date), foreign key(num) references student(num))'
cur.execute(sql)
db.commit()
cur.close()
# 학생 점수 테이블 생성 

cur = db.cursor()
sql = 'create or replace view vw_left as select m.num, m.name, m.hp, m.email, m.address, p.java, p.python, p.c, p.total, p.avg from kdt.student as m left outer join kdt.grade as p on m.num = p.num;'
cur.execute(sql)
db.commit()
cur.close()
# 학생 정보와 점수가 있는 데이터의  테이블 생성 

cur = db.cursor()
sql ='create or replace view vw as  select m.num ,m.name ,m.hp ,m.email ,m.address ,p.java ,p.python ,p.c,p.total,p.avg from kdt.student as m inner join kdt.grade as p on m.num = p.num;'
cur.execute(sql)
db.commit()
cur.close()
# 학생 점수가 없어도 합쳐진 데이터의  테이블 생성 

In [2]:
class DTO: # 정보 테이블 DTO
    def __init__ (self,num,name,hp,email,address):
        self.num = num
        self.name = name
        self.hp = hp
        self.email = email
        self.address = address
        
    def setNum(self,num):
        self.num = num
    
    def getNum(self):
        return self.num
        
    def setName(self,name):
        self.name = name
    
    def getName(self):
        return self.name
    
    def setHp(self,hp):
        self.hp = hp
    
    def getHp(self):
        return self.hp
    
    def setEmail(self,email):
        self.email = email
    
    def getEmail(self):
        return self.email
    
    def setAddress(self,address):
        self.address = address
    
    def getAddress(self):
        return self.address
    
    def setEnterdate(self,enterdate):
        self.enterdate = enterdate
    
    def getEnterdate(self):
        return self.enterdate
    
    def print_word(self):

        print(f'학번 : {self.num} | 이름 : {self.name}  번호 : {self.hp} | 이메일 : {self.email} | 주소 : {self.address} | 등록날짜 : {self.enterdate} ')

In [3]:
class GradeDTO: # 점수 테이블 DTO
    def __init__ (self,num,java,python,c,total,avg):
        self.num = num
        self.java = java
        self.python = python
        self.c = c
        self.total = total
        self.avg = avg
    
    def setNum(self,num):
        self.num = num
    
    def getNum(self):
         return self.num
        
    def setJava(self,java):
        self.java = java
    
    def getJava(self):
         return self.java
        
    def setPython(self,python):
        self.python = python
    
    def getPython(self):
         return self.python

    def setC(self,c):
        self.c = c
    
    def getC(self):
         return self.c
    
    def setTotal(self,total):
        self.total = c
    
    def getTotal(self):
         return self.total

    def setAvg(self,avg):
        self.avg = avg
    
    def getAvg(self):
         return self.avg
    
        

In [4]:
import MySQLdb 

class Dao:
    def __init__(self):
        self.db = None
        
    def connect(self):
        self.db = MySQLdb.connect('localhost','root','1234','kdt')
    
    def disconnect(self):
        self.db.close()
        
    def insert1(self,info): #학생 정보 입력 DAO
        self.connect()
        cur = self.db.cursor()
        sql = 'insert into student(num,name,hp,email,address) values (%s, %s, %s, %s, %s)'
        data = (info.getNum(),info.getName(),info.getHp(),info.getEmail(),info.getAddress())
        cur.execute(sql,data)
        self.db.commit()
        self.disconnect()
        
    def insert2(self,info): #학생 점수 입력 DAO  #입력시 total과 avg 변수를 추가해서 자동입력
        self.connect()
        cur = self.db.cursor()
        sql = 'insert into grade(num,java,python,c,total,avg) values (%s, %s, %s, %s, %s, %s)'
        data = (info.getNum(),info.getJava(),info.getPython(),info.getC(),info.getTotal(),info.getAvg())
        cur.execute(sql,data)
        self.db.commit()
        self.disconnect()
        
        
    def selectAll(self): 
            self.connect()
            cur = self.db.cursor(MySQLdb.cursors.DictCursor)
            sql = 'select *,rank() over(order by avg desc) As ranking from vw_left'
            cur.execute(sql)
            row = cur.fetchone()

            while row is not None:
                print(row)
                row = cur.fetchone()
    # 전체 출력시 rank함수를 사용하여, 순위를 보여줌  (기준: 평균값)
    # left outer join으로 점수값이 있어도 출력, 없어도 출력 
                
           
    def search(self,info):
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)
        sql = 'select * from vw_left where name = %s'
        data = (info,)
        cur.execute(sql,data)
        row = cur.fetchone()
        return row
        self.disconnect()
        
    #left outer join을 한 view에서 검색
    
    def edit_Info(self,before_name,column,new):
        self.connect()
        cur =self.db.cursor()
        sql = 'update student set ' + column  +' = %s where name = %s'
        data =(new,before_name)
        row = cur.execute(sql,data)
        self.db.commit()
        if row > 0 :
            print('수정 완료')
        else:
            print('다시 입력하세요')
    # 수정할 데이터의 분류를 column으로 받아서 수정 
    # 여기서 column은 데이터 필드 이름 (정보 데이터만)
            
    def delete_Info(self,name):
        self.connect()   
        cur = self.db.cursor()
        sql = 'delete from grade where num = (select num from student where name = %s)'
        data =(name,)
        row2 = cur.execute(sql,data)
        
        cur = self.db.cursor()
        sql = 'delete from student where name = %s'
        data =(name,)
        row1 = cur.execute(sql,data)
        self.db.commit()
        if row1 > 0:
            print(f'{name}님의 정보가 삭제 되었습니다.')
        else:
            print('삭제 실패!')    
        #학생 정보 수정
        #학번이라는 공통된 데이터가 있지만, 학생이름으로 수정하는 것이 더 편할 것 같아
        # 서브 쿼리 작성 where문
        # sql문으로 grade 자체의 정보 삭제
    
    def delete_Info2(self,name):
        self.connect()
        cur = self.db.cursor()
        sql = 'delete from grade where num =  (select num from student where name = %s)'
        data =(name,)
        row = cur.execute(sql,data)
        self.db.commit()
        if row > 0:
            print(f'{name}님의 정보가 삭제 되었습니다.')
        else:
            print('삭제 실패!')     
    
        #학생 점수 수정
        #학번이라는 공통된 데이터가 있지만, 학생이름으로 수정하는 것이 더 편할 것 같아
        # 서브 쿼리 작성 where문
        # sql문으로 grade 자체의 정보 삭제

    def edit_Info2(self,before,new1,new2,new3,total,avg):
        self.connect()
        cur =self.db.cursor()
        sql = 'update vw set java= %s, python=%s, c=%s, total=%s,avg=%s where name = %s'
        data =(new1,new2,new3,total,avg,before)
        row = cur.execute(sql,data)
        self.db.commit()
        if row > 0 :
            print('수정 완료')
        else:
            print('다시 입력하세요')
            
        #점수 수정
        #vw는 inner join을 해서 update를 사용할 수 있어서 사용
        #수정시 점수 3개 다시 입력을 해야함.
    

In [5]:
class Service:
    def __init__(self):
        self.dao = Dao()
        
    def insertInfo(self):
        num = input('학번')
        name = input('이름')
        hp = input('전화번호')
        email = input('이메일')
        address = input('주소')
        info = DTO(num,name,hp,email,address)
        self.dao.insert1(info)
    
    #정보 입력받는 곳
    #정보 DTO에 넣어서 DAO 중 정보에 해당하는 곳에 넣음
    
    def insertInfo2(self):
        num = input('학번')
        java = int(input('자바점수'))
        python = int(input('파이썬'))
        c = int(input('C언어점수'))
        total = java+python+c
        avg=int((java+python+c)/3)
        info = GradeDTO(num,java,python,c,total,avg)
        self.dao.insert2(info)
    
    #점수 입력받는 곳
    #점수 DTO에 넣어서 DAO 중 정보에 해당하는 곳에 넣음
        
    def printAll(self):
        datas = self.dao.selectAll()
    

    def searchInfo(self):
        search = input('검색할 이름 입력')
        info = self.dao.search(search)
        if info != None:    
            print(info)
        else:
            print('입력하신 이름을 찾을 수 없습니다.')


    def editInfo(self):
        before = input('수정할 사람의 이름을 입력하세요.')
        row = self.dao.search(before)
        print(row)
        column = input('수정할 학생정보를 입력하세요\n ex) name,hp,email')
        new = input('새로운 정보를 입력하세요')
        self.dao.edit_Info(before,column,new)
        
    # column이란 변수를 넣어서 학번,이름,전화번호 등 변경할 수 있게 잡음 
    
    def deleteInfo(self):
        delete_data = input('삭제할 사람의 이름을 입력하세요')
        self.dao.delete_Info(delete_data)
    # 학생정보 삭제 기능
    
    def editInfo2(self):
        before = input('수정할 사람의 이름을 입력하세요.')
        row = self.dao.search(before)
        print(row)
        new1 = int(input('java를 입력하세요'))
        new2 = int(input('python를 입력하세요'))
        new3 = int(input('C언어를 입력하세요'))
        total= (new1+new2+new3)
        avg = int((new1+new2+new3)/3)
        self.dao.edit_Info2(before,new1,new2,new3,total,avg)
    #학생 점수 수정 기능
    #점수를 수정받을 곳을 입력받는 곳 
    # total,avg를 재 계산해서 삽입

    def deleteInfo2(self):
        delete_data = input('삭제할 사람의 이름을 입력하세요')
        self.dao.delete_Info2(delete_data)

In [6]:
class Menu:
    def __init__(self):
        self.service = Service()
        
    def run(self):
        while True:
            print('==============학생부====================')
            menu = int(input('1.학생정보입력\n2.학생정보전체\n3.학생정보검색\n4.학생정보 삭제\n5.학생정보 수정\n6.학생점수 입력\n7.학생점수 수정\n8.학생점수 삭제\n9.프로그램 종료'))
                       
            if menu == 1:
                self.service.insertInfo()
                print('완료')
            elif menu == 2:
                self.service.printAll()
            elif menu ==3:
                self.service.searchInfo()
            elif menu ==4:
                self.service.deleteInfo()
            elif menu ==5:
                self.service.editInfo()
            elif menu ==6:
                self.service.insertInfo2()
            elif menu ==7:
                self.service.editInfo2()
            elif menu ==8:
                self.service.deleteInfo2()
            else:
                break
                
            
                             

In [7]:
a = Menu()
a.run()

==============학생부====================
1.학생정보입력
2.학생정보전체
3.학생정보검색
4.학생정보 삭제
5.학생정보 수정
6.학생점수 입력
7.학생점수 수정
8.학생점수 삭제
9.프로그램 종료1
학번2016
이름최영현
전화번호010
이메일1
주소2
완료
==============학생부====================
1.학생정보입력
2.학생정보전체
3.학생정보검색
4.학생정보 삭제
5.학생정보 수정
6.학생점수 입력
7.학생점수 수정
8.학생점수 삭제
9.프로그램 종료2
{'num': '2016', 'name': '최영현', 'hp': '010', 'email': '1', 'address': '2', 'java': None, 'python': None, 'c': None, 'total': None, 'avg': None, 'ranking': 1}
==============학생부====================
1.학생정보입력
2.학생정보전체
3.학생정보검색
4.학생정보 삭제
5.학생정보 수정
6.학생점수 입력
7.학생점수 수정
8.학생점수 삭제
9.프로그램 종료3
검색할 이름 입력최영현
{'num': '2016', 'name': '최영현', 'hp': '010', 'email': '1', 'address': '2', 'java': None, 'python': None, 'c': None, 'total': None, 'avg': None}
==============학생부====================
1.학생정보입력
2.학생정보전체
3.학생정보검색
4.학생정보 삭제
5.학생정보 수정
6.학생점수 입력
7.학생점수 수정
8.학생점수 삭제
9.프로그램 종료4
삭제할 사람의 이름을 입력하세요최영현
최영현님의 정보가 삭제 되었습니다.
==============학생부====================
1.학생정보입력
2.학생정보전체
3.학생정보검색
4.학생정보 삭제
5.학생정보 수정
6.학생점수 입력
7.학생점수 수정
8.학생

###### 

33
